---

Created for [learn-investments.rice-business.org](https://learn-investments.rice-business.org)
    
By [Kerry Back](https://kerryback.com) and [Kevin Crotty](https://kevin-crotty.com)
    
Jones Graduate School of Business, Rice University

---


# EXAMPLE DATA

In [1]:
S = 50                          # initial price of underlying
K = 50                          # strike price
sigma = 0.4                     # volatiliy
r = 0.01                        # risk-free rate per period
time_to_ex = 1                  # years to ex-dividend date
time_ex_to_mat = 1              # years from ex-dividend date to maturity

T = time_to_ex + time_ex_to_mat # total time to maturity

# FUNCTION

In [5]:

import numpy as np
from scipy.stats import norm
from scipy.stats import multivariate_normal as mnorm
from scipy.optimize import fsolve


def callBS(S, time):
    if S<=0: return 0
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * time) / (sigma * np.sqrt(time))
    d2 = d1 - sigma * np.sqrt(T)
    return S * norm.cdf(d1) - np.exp(-r * time) * K * norm.cdf(d2)


def call_delta(S, time):
    if S <= 0: return 0
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * time) / (sigma * np.sqrt(time))
    return norm.cdf(d1)

def criticalS(div):

    def f(x):
        return x - K - callBS(x - div, time_ex_to_mat)

    def fprime(x):
        return 1 - call_delta(x - div, time_ex_to_mat)

    return fsolve(f, x0=1.5 * K, fprime=fprime).item()


def binorm(a, b, corr):
    cov = [[1, corr], [corr, 1]]
    return mnorm.cdf(x=[a, b], mean=[0, 0], cov=cov)

def American(div):

    # underlying value stripped of dividend
    # equals forward price of underlying
    Z = S - np.exp(-r * time_to_ex) * div 

    # sometimes it will never be optimal to exercise early,
    # so we just use Black-Scholes on the forward price
    if div <= (1 - np.exp(-r * time_ex_to_mat)) * K:
        return callBS(Z, T)
    
    # otherwise, we find the critical price    
    Sstar = criticalS(div)

    # then we use a variation of Black-Scholes
    # see Back, A Course in Derivative Securities, 2006

    Zstar = Sstar - div
    d1 = (np.log(Z / Zstar) + (r + 0.5 * sigma**2) * time_to_ex)
    d1 /= (sigma * np.sqrt(time_to_ex))
    d2 = d1 - sigma * np.sqrt(time_to_ex)
    N1 = norm.cdf(d1)
    N2 = norm.cdf(d2)

    d1_two = (np.log(Z / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2_two = d1_two - sigma * np.sqrt(T)
    corr = -np.sqrt(time_to_ex / T)
    M1 = binorm(-d1, d1_two, corr)
    M2 = binorm(-d2, d2_two, corr)

    return (
        Z * (N1 + M1)
        + np.exp(-r * time_to_ex) * (div - K) * N2
        - np.exp(-r * T) * K * M2
    )

# FIGURE 1

In [9]:
import plotly.graph_objects as go

grid = np.arange(0.1, 10.1, 0.1)
grid = grid[grid>(1 - np.exp(-r * time_ex_to_mat)) * K]


string="critical price is $%{y:.2f}<br>when dividend is $%{x:.2f}<extra></extra>"
trace = go.Scatter(
    x=grid,
    y=[criticalS(x) for x in grid],
    mode="lines",
    hovertemplate=string
)

fig = go.Figure(trace)

fig.update_layout(
    xaxis_tickprefix="$",
    yaxis_tickprefix="$",
    xaxis_tickformat=".2f",
    yaxis_tickformat=".0f",
    xaxis_title="Dividend",
    yaxis_title="Critical Price of Underlying",
    template="plotly_white"
)
fig.show()



# FIGURE 2

In [16]:
grid = np.arange(0.1, 10.1, 0.1)

trace1 = go.Scatter(
    x=grid,
    y=[callBS(S-np.exp(-r * time_to_ex)*x, T) for x in grid],
    mode="lines",
    name="European",
    hovertemplate="European value = $%{y:.2f}<extra></extra>"
)

trace2 = go.Scatter(
    x=grid,
    y=[American(x) for x in grid],
    mode="lines",
    name="American",
    hovertemplate="American value = $%{y:.2f}<extra></extra>",
    legendrank=1
)

fig = go.Figure(trace1)
fig.add_trace(trace2)

fig.update_layout(
    xaxis_tickprefix="$",
    yaxis_tickprefix="$",
    xaxis_tickformat=".2f",
    yaxis_tickformat=".0f",
    xaxis_title="Dividend",
    yaxis_title="Option Value",
    hovermode="x unified",
    template="plotly_white",
    legend=dict(
        x=0.99,
        xanchor="right",
        y=0.99,
        yanchor="top"
    )
)

fig.show()